In [ ]:
# Test Pareto Optimizer
import sys

sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

import pandas as pd
import json
from typing import Dict, Any, List
import numpy as np
from datetime import datetime, timedelta
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs, Trial
from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer
from robyn.data.entities.enums import (
    DependentVarType,
    PaidMediaSigns,
    OrganicSigns,
    ContextSigns,
)

from robyn.tutorials.utils.data_mapper import (
    import_output_models,
    import_input_collect,
    load_data_from_json,
)

In [ ]:
# Load data from JSON
inputCollect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/test_Pareto_2000_iterations_5_trials_InputCollect.json"
)
outputModel = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/test_Pareto_2000_iterations_5_trials_OutputModels.json"
)
input_collect = import_input_collect(inputCollect)
model_outputs = import_output_models(outputModel)
display((model_outputs.hyper_bound_ng))

In [ ]:
mmm_data = input_collect["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs

model_outputs = model_outputs
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = input_collect["hyperparameters"]
# display(hyperparameters)

featurized_mmm_data = input_collect["featurized_mmm_data"]

holidays_data = input_collect["holidays_data"]

In [ ]:
# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(
    mmm_data, model_outputs, hyperparameters, featurized_mmm_data, holidays_data
)

In [ ]:
# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=100)

In [ ]:
# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto fronts: {pareto_result.pareto_solutions}")
print(
    f"MediaVecCollect: {pareto_result.media_vec_collect.shape, pareto_result.media_vec_collect}"
)
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param)

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg)
print("\result Calibration:")
print(pareto_result.result_calibration)
print("\nx Decomp Vec Collect:")
print(pareto_result.x_decomp_vec_collect.shape, pareto_result.x_decomp_vec_collect)
print("\nCarryover percentage all:")
print(pareto_result.df_caov_pct_all.shape, pareto_result.df_caov_pct_all)
print("\Plot Data Collected")
# print("NUMBER OF PLOTS Data collected for:", len(pareto_result.plot_data_collect["3_206_6"]))
# print("Plot data for solid 3_206_6", pareto_result.plot_data_collect["3_206_6"])

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(
    pareto_result.pareto_fronts, int
), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")

In [ ]:
print(pareto_result.x_decomp_agg[pareto_result.x_decomp_agg["sol_id"] == "5_221_9"])

# Only export this to a pickle file if you plan to use the results and the mmm_data in another notebook

In [ ]:
import pickle

with open("pareto_result.pkl", "wb") as f:
    pickle.dump(pareto_result, f)
with open("mmmdata.pkl", "wb") as f:
    pickle.dump(mmm_data, f)
with open("holidays_data.pkl", "wb") as f:
    pickle.dump(holidays_data, f)
with open("featurized_mmm.pkl", "wb") as f:
    pickle.dump(featurized_mmm_data, f)